# Clipping Sentinel-2 images

You already have code to help download Sentinel-2 imagery via the sentinelsat API (see Week 3). 

This tutorial code now helps you to process those images. 

Firstly, you need to place your sentinel2 .jp2 images in the same folder as this notebook (or use a path navigation to help load them - whichever you prefer).

Then they can be loaded, clipped using a `rasterio` mask, and written out as follows.

In [13]:
import os
import rasterio
from matplotlib import pyplot as plt
from rasterio.mask import mask

# Insert the name of your files here! 
# They must be in the same folder as this notebook. 
bands = [
    "T18SUH_20220110T155631_B02.jp2", #blue   
    "T18SUH_20220110T155631_B03.jp2", #green
    "T18SUH_20220110T155631_B04.jp2", #red
]

for band in bands:

    # Open our image
    my_image = rasterio.open(band)
    band1 = my_image.read(1)

    # We can then specify the part of the image we want to clip using geographic coordinates. 
    xmin = 319677
    ymin = 4295835
    xmax = 323391
    ymax = 4298996

    # Specify our geojson boundary to clip the image to. 
    my_geojson = [{
        "type": "Polygon", #let's define our geometry type, which as we have a square, is a polygon.
        "coordinates": [ #Here are our actual geometry coordinates
          [
            [
              xmin,
              ymin
            ],
            [
              xmax,
              ymin
            ],
            [
              xmax,
              ymax
            ],
            [
              xmin,
              ymax
            ],
            [
              xmin,
              ymin
            ]
          ],
        ]
      }]

    # Now we can apply the rasterio mask 
    # We specify the function needs to crop (via crop=True)
    with rasterio.open(band) as img:
        clipped, transform = mask(img, my_geojson, crop=True)

    # Great! Now you've clipped the image. 
    # We just finally need to gather the correct updated metadata to write the new image
        
    # Let's first copy the metadata from the original ratserio object
    meta = my_image.meta.copy()

    # We will need to update this metadata, and provide the new clipped boundaries
    meta.update(
        {

            "transform": transform,
            "height":clipped.shape[1],
            "width":clipped.shape[2]
        }
    )

    # Now we can add a filename for our new clipped image
    # Note: We can reuse the band name
    filename_out = '{}_clipped.tif'.format(band) 

    # Finally, we can write the new image to a GeoTIFF (.tif)
    with rasterio.open(filename_out, 'w', **meta) as dst:
        dst.write(clipped)

    print('Writing complete for {}'.format(filename_out))


Writing complete for T18SUH_20220110T155631_B02.jp2_clipped.tif


RasterioIOError: T18SUH_20220110T155631_B03.jp2: No such file or directory